In [1]:
import cv2
import numpy as np

In [2]:
def origin(rec_vertices):
    return [sum(x)/ 2 for x in zip(rect_vertices[0],rect_vertices[2])]

In [3]:
def matrix(rect_vectices, matrix_transformation):
    G = np.array(rect_vertices)
    one = np.ones((G.shape[0],1))
    GConcate = np.concatenate((G,one),axis=1)
    GTrans = np.transpose(matrix_transformation.dot(np.transpose(GConcate)))
    matrix_points = [[int(element) for element in sublist] for sublist in GTrans[:, :2].tolist()]
    return [tuple(x) for x in matrix_points]

In [9]:
def transformation(rect_vertices, delta_x=0, delta_y=0, ratio_x=1.0, ratio_y=1.0, angle=0):
    center = origin(rect_vertices)
    radian = np.radians(angle).astype(np.float64)
    cos = np.cos(radian)
    sin = np.sin(radian)
    alpha = ratio_x * cos
    beta = ratio_y * sin
    S = np.array(((alpha, beta, (1 - alpha) * center[0] - beta * center[1] + delta_x),
                  (-beta, alpha, beta * center[0] + (1 - alpha) * center[1] + delta_y),
                  (0,0,1)
    ))
    return matrix(rect_vertices, S)

In [5]:
def mouse_callback(event, x, y, flags, param):
    # Note: IO -> mutable
    global drawing, p1, p2, img, rect_vertices, center
    if event == cv2.EVENT_LBUTTONDOWN:
        rect_vertices = [(0, 0), (0, 0), (0, 0), (0, 0)]
        img = np.ones((HEIGHT, WIDTH, 3), np.uint8) * 255
        drawing = True
        p1 = (x, y)
        
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        p2 = (x, y)
        cv2.rectangle(img, p1, p2, (0, 255, 0), 2)
        cv2.imshow('Image', img)
        rect_vertices = [p1, (p2[0], p1[1]), p2, (p1[0], p2[1])]
        center = origin(rect_vertices)

In [6]:
def create_whiteboard():
    # Note: thực hiện tranformation thì functional
    global rect_vertices, img, key, center
    while True:
        cv2.setMouseCallback('Image', mouse_callback)
        cv2.imshow('Image', img)
        key = cv2.waitKey(1)
        if key == ord('p'):
            print([sum(x)/2 for x in zip(rect_vertices[0], rect_vertices[2])])
        if key == 27:  # Press Esc key to exit
            break
        if key == ord('t'):
            BLUE = (255, 0, 0)
            delta_x, delta_y = map(float, input("Type delta_x delta_y (e.g: 12.3 -231.2): ").split())
            rect_vertices = transformation(rect_vertices, delta_x=delta_x, delta_y=delta_y)
            cv2.polylines(img, np.array([rect_vertices]), True, BLUE, 3)
            cv2.imshow('Image', img)
            continue
        if key == ord('r'):
            RED = (0, 0, 255)
            angle = float(input("Type angle (e.g: 90): "))
            rect_vertices = transformation(rect_vertices, angle=angle)
            cv2.polylines(img, np.array([rect_vertices]), True, RED, 3)
            cv2.imshow('Image', img)
            continue
        if key == ord('s'):
            PURPLE = (255, 13, 255)
            ratio_x, ratio_y = map(float, input("Type scale for x, y(e.g: 0.6 2.1) ").split())
            rect_vertices = transformation(rect_vertices, ratio_x=ratio_x, ratio_y=ratio_y)
            cv2.polylines(img, np.array([rect_vertices]), True, PURPLE, 3)
            cv2.imshow('Image', img)
            continue
    cv2.destroyAllWindows()
        

In [7]:
def main():
    global drawing, p1, p2, HEIGHT, WIDTH, img, rect_vertices, center
    drawing = False
    p1 = np.array((-1, -1))
    p2 = np.array((-1, -1))
    HEIGHT, WIDTH = 1980, 1720
    img = np.ones((HEIGHT, WIDTH, 3), dtype=np.uint8) * 255
    rect_vertices = [(0, 0), (0, 0), (0, 0), (0, 0)]
    center = [sum(x)/2 for x in zip(rect_vertices[0],rect_vertices[2])]
    cv2.namedWindow('Image')
    create_whiteboard()

In [8]:
main() # Run me

Type delta_x delta_y (e.g: 12.3 -231.2): 20 20
1.0 1.0
Type angle (e.g: 90): 20
1.0 1.0
